# Conversion to ONNX
ONNX is a different format for running machine learning models. The ONNX format is much faster on CPU, sometimes 5 times as fast as PyTorch!

While the EAWSW model is designed to be small, accurate and accessible, for some people it's still too much to run...

Hosting the model as a free service for players is an option. An ONNX version of the model allows us to host the model on CPU yet have faster response times! Given that the model is made in a time with chip shortage, running on hardware I already have inside a server is efficient, scalable and cheaper.

An important note is that ONNX doesn't execute logic by itself, and you have to do that yourself, `onnx_model_manager.py` intends to deal with this for us.

In [1]:
%load_ext autoreload
%autoreload 2

from model_utils import train_model, split_data, split_branches, get_model, set_pretrained_model_dropout, get_dataset, ModelSeeder
from config import Config
import json
import matplotlib.pyplot as plt
%matplotlib inline
import math
import random
import time
import onnx
import logging
from onnx_model_manager import OnnxModelManager
from onnxruntime.quantization import quantize_dynamic, QuantType
import os
import datasets
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from model_manager import ModelManager

In [2]:
saved_model_path = os.path.join("models", "awsw_mixed")
saved_model_onnx_path = os.path.join("models", "awsw_onnx")
if not os.path.exists(os.path.join(saved_model_path, "special_tokens_map.json")):
    print("Copying config files from huggingface (needed for conversion)... WARNING: this assumes the structure of the model isn't changed!")
    !cd $saved_model_path && git clone https://huggingface.co/EleutherAI/gpt-neo-125M
    !cp -n $saved_model_path/gpt-neo-125M/* $saved_model_path
    !rm -rf $saved_model_path/gpt-neo-125M
if not os.path.exists(os.path.join(saved_model_onnx_path, "model.onnx")):
    !python3 -m transformers.onnx --model=$saved_model_path --feature=causal-lm-with-past $saved_model_onnx_path

In [3]:
def optimize_onnx():
    model_quant = os.path.join(saved_model_onnx_path, "model_quant.onnx")
    if not os.path.exists(model_quant):
        model_fp32 = os.path.join(saved_model_onnx_path, "model.onnx")
        model_opt = os.path.join(saved_model_onnx_path, "model-opt.onnx")
        quantized_model = quantize_dynamic(model_fp32, model_quant, weight_type=QuantType.QInt8)
        #!rm $model_opt
optimize_onnx()

In [4]:
# Tell pytorch to run this model on the GPU.
device_name = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(device_name)

onnx_model_manager = OnnxModelManager(os.path.join(saved_model_onnx_path, "model-opt.onnx"))
onnx_model_manager_quant = OnnxModelManager(os.path.join(saved_model_onnx_path, "model_quant.onnx"))
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neo-125M')
model = AutoModelForCausalLM.from_pretrained(saved_model_path)
model.to(device)
model.eval()
model_manager = ModelManager(model=model, tokenizer=tokenizer, device=device)
print(f"Pretrained model loaded on {device_name}")

Pretrained model loaded on cuda:0


In [5]:
prompt = "In my dreams, I'm a dragon"
for i in range(10):
    print("ONNX:", onnx_model_manager.say_raw(prompt, do_sample=True))
    print("ONNX (Quantized):", onnx_model_manager_quant.say_raw(prompt, do_sample=True))
    print("PyTorch:", model_manager.say_raw(prompt, 50, 0.7))
    print('-' * 100)

ONNX: In my dreams, I'm a dragon. I've got hands, feet, and a lot of hair. I just don't think I can do it anymore."<p>I'm not sure what to think anymore."<p><msg>c "I'm not going to let this go, are you, and the rest of this is about my personal safety."<d><scn>np2x<msg>Ad "That is, unless I can prove it."<|endoftext|>
ONNX (Quantized): In my dreams, I'm a dragon. I'm a dragon, and I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I
PyTorch: In my dreams, I'm a dragon. I'm a dragon, I'm a dragon, I'm a dragon, I'm a dragon. I'm a dragon, I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'm a dragon. I'

# Testing

We created a few past (for context) + present prompts (player input) and see the different reactions. This way, we can test the models across different iterations.
The first test involves a old prompt to compare the pre-trained model with the one trained on AWSW. Did it manage to store it's data well? Is it able to write down things that have nothing to do with AWSW? (So we know we didn't overfit).

**This test generates boring and repetetive** replies! It's because we use no good sampling algorithm, but it does give us a indication of what the model has learned!

In [6]:
prompts = [
    ('<p><msg>c "Hey Remy!"<d><scn>park2<msg>Ry "Hey!"', "How are you?"),
    ('<p><msg>c "I was with Lorem today."<d><scn>park2<msg>Ad "Very nice."', "What do you think of Lorem?"),
    ('<p><msg>m "In Tatsu park, Adine and I sat down."', "Oh my god, Adine. What is this?"),
    ('<p><msg>m "I sat down on a chair in Anna\'s lab."', "What will we do here?"),
]

for (past, prompt) in prompts:
    print(f"Prompt: {prompt}")
    reply = model_manager.say(past, prompt)
    print(f"[Pytorch] Reply: {reply}\n")
    reply = onnx_model_manager.say(past, prompt)
    print(f"[ONNX] Reply: {reply}\n")
    reply = onnx_model_manager_quant.say(past, prompt)
    print(f"[ONNX Quantized] Reply: {reply}\n")
    print("-" * 10)

Prompt: How are you?
[Pytorch] Reply: park2<msg>Ry "I'm fine. I'm just a little busy."<p><msg>c "I'm not sure if I should take any visitors."<d><scn>park2<msg>Ry "No, I'm not going to be rude or anything. I just want to get this over with."<p><msg>c "I'm not sure if I should take any

[ONNX] Reply: park2<msg>Ry "I'm fine. I'm just a little busy."<p><msg>c "I'm not sure if I should take any visitors."<d><scn>park2<msg>Ry "No, I'm not going to be rude or anything. I just want to get it over with."<p><msg>c "I'm not sure if I should take any visitors."<d><scn>park2<msg>Ry "No, I'm not going to be rude or anything. I just want to get it over with."<p><msg>c

[ONNX Quantized] Reply: park2<msg>Ry "I'm fine, I'll just have to be careful."<p><msg>c "I'm fine."<d><scn>park2<msg>Ry "I'm fine, I'll just have to be careful."<|endoftext|>

----------
Prompt: What do you think of Lorem?
[Pytorch] Reply: park2<msg>Ad "I think he's a good student."<p><msg>c "I'm not sure, I am not a linguist. Are you?

# Sampling test

This is gonna be interesting!

In [7]:
for (past, prompt) in prompts:
    print(f"Prompt: {prompt}")
    reply = model_manager.say(past, prompt, top_k = 50, top_p = 0.7)
    print(f"[Pytorch] Reply: {reply}\n")
    reply = onnx_model_manager.say(past, prompt, do_sample = True)
    print(f"[ONNX] Reply: {reply}\n")
    reply = onnx_model_manager_quant.say(past, prompt, do_sample = True)
    print(f"[ONNX Quantized] Reply: {reply}\n")
    print("-" * 10)

Prompt: How are you?
[Pytorch] Reply: park2<msg>Ry "I'm fine. I just have a few questions."<d><scn>park2<msg>Ry "I'm sure you have a point."<p><msg>c "I'm sorry, I don't know how to say this."<d><scn>park2<msg>Ry "I'm sorry, I don't know how to say this. I

[ONNX] Reply: park2<msg>Ry "I was just kidding."<p><msg>c "Hey [player_ name]."<d><scn>park2<msg>Ry "Hey."<|endoftext|>

[ONNX Quantized] Reply: park2<msg>Ry "It's not that easy. It seems like I'm in a rush."<p><msg>c "It's not a problem. You're right."<d><scn>park2<msg>Ry "Well, I guess I'll just have another to slip up about."<|endoftext|>

----------
Prompt: What do you think of Lorem?
[Pytorch] Reply: park2<msg>Ad "I'm not sure, but he's a good fellow."<p><msg>c "I think he's a good student."<d><scn>park2<msg>Ad "I'm not sure, but you do have a point."<p><msg>c "I'm not sure, but you do have a point

[ONNX] Reply: park2<msg>Ad "It's pretty important you get your tail to the right edge, right?"<p><msg>c "I'm not. I don't know wha

# RP test
Testing out the injected roleplay actions

In [8]:
test_rps = [
    "Visit Lorem",
    "Meet with Lorem",
    "Visit Adine",
    "Fight Maverick",
    "Fight Adine",
    "Attack Adine"
]
for rp in test_rps:
    print(f'[Pytorch] {rp} -> {model_manager.say("", rp, top_k = 50, top_p = 0.7)}')
    print(f'[ONNX] {rp} -> {onnx_model_manager.say("", rp, do_sample = True)}')
    print(f'[ONNX Quantized] {rp} -> {onnx_model_manager_quant.say("", rp, do_sample = True)}')
    print("-" * 10)

[Pytorch] Visit Lorem -> loremapt<msg>Lo "Hey, Lorem."<d><scn>loremapt<msg>Lo "Hey, Lorem."<d><scn>loremapt<msg>Lo "Hey, Lorem."<d><scn>loremapt<msg>Lo "Hey, Lorem."<d><scn>loremapt<msg>Lo "Hey, Lorem."<|endoftext|>
[ONNX] Visit Lorem -> loremapt<msg>Lo "Oh. Good morning."<|endoftext|>
[ONNX Quantized] Visit Lorem -> lorem<msg>Lo "Oh, I'm just messing around."<|endoftext|>
----------
[Pytorch] Meet with Lorem -> loremapt<msg>Lo "Hey [player_name]!"<|endoftext|>
[ONNX] Meet with Lorem -> loremapt<msg>Lo "Oh, [Oh, [I can't remember anything about him, but I think it's pretty important for the police to have a nice evening."<|endoftext|>
[ONNX Quantized] Meet with Lorem -> lorem<msg>Lo "Oh. I see. Thank's for taking the time to pay my visit."<|endoftext|>
----------
[Pytorch] Visit Adine -> adineapt<msg>Ad "Oh, I see."<|endoftext|>
[ONNX] Visit Adine -> adineapt<msg>Ad "Hey, [player_name]!"<|endoftext|>
[ONNX Quantized] Visit Adine -> adineapt<msg>Ad "Oh, you're in luck. I have something 